[View in Colaboratory](https://colab.research.google.com/github/vmartinezalvarez/Deep-Learning-by-Google_UDACITY/blob/master/3_regularization.ipynb)

Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [0]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from hashlib import sha1

First reload the data we generated in `1_notmnist.ipynb`.

In [26]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset1 = save['train_dataset']
  train_labels1 = save['train_labels']
  valid_dataset1 = save['valid_dataset']
  valid_labels1 = save['valid_labels']
  test_dataset1 = save['test_dataset']
  test_labels1 = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset1.shape, train_labels1.shape)
  print('Validation set', valid_dataset1.shape, valid_labels1.shape)
  print('Test set', test_dataset1.shape, test_labels1.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [27]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset1, train_labels1)
valid_dataset, valid_labels = reformat(valid_dataset1, valid_labels1)
test_dataset, test_labels = reformat(test_dataset1, test_labels1)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [0]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Logistic Classifier Sklearn

In [57]:
a= train_dataset[:10000,:]
print(a.shape)
b = train_labels[:10000]
train_dataset.shape

(10000, 784)


(200000, 784)

In [32]:
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from sklearn import metrics


logit_clf = LogisticRegression(penalty='l2')

logit_clf.fit(train_dataset[:10000,:], train_labels1[:10000])

predicted = logit_clf.predict(test_dataset)
print ('accuracy', accuracy((np.arange(num_labels) == predicted[:,None]).astype(np.float32), test_labels), '%')


accuracy 86.02 %


## Shallow Multinomial Logistic Regression

In [35]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  l2_regularizer = tf.nn.l2_loss(weights)
  loss += 5e-4 * l2_regularizer
  
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [36]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16.731560
Training accuracy: 16.8%
Validation accuracy: 19.8%
Loss at step 100: 3.533545
Training accuracy: 73.1%
Validation accuracy: 71.6%
Loss at step 200: 2.960108
Training accuracy: 76.1%
Validation accuracy: 74.1%
Loss at step 300: 2.609129
Training accuracy: 77.4%
Validation accuracy: 75.1%
Loss at step 400: 2.352725
Training accuracy: 78.3%
Validation accuracy: 75.4%
Loss at step 500: 2.148804
Training accuracy: 79.0%
Validation accuracy: 75.6%
Loss at step 600: 1.979333
Training accuracy: 79.6%
Validation accuracy: 75.7%
Loss at step 700: 1.835155
Training accuracy: 80.3%
Validation accuracy: 75.9%
Loss at step 800: 1.710586
Training accuracy: 80.8%
Validation accuracy: 76.2%
Test accuracy: 83.0%


## Stochastic Gradient Descent

In [0]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  l2_regularizer = tf.nn.l2_loss(weights)
  loss += 10e-4 * l2_regularizer
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
   
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [49]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 22.273397
Minibatch accuracy: 8.6%
Validation accuracy: 13.6%
Minibatch loss at step 500: 3.287861
Minibatch accuracy: 71.9%
Validation accuracy: 76.4%
Minibatch loss at step 1000: 1.928576
Minibatch accuracy: 74.2%
Validation accuracy: 78.1%
Minibatch loss at step 1500: 1.214303
Minibatch accuracy: 82.0%
Validation accuracy: 80.2%
Minibatch loss at step 2000: 0.978054
Minibatch accuracy: 82.0%
Validation accuracy: 81.5%
Minibatch loss at step 2500: 0.725850
Minibatch accuracy: 85.9%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 0.731505
Minibatch accuracy: 85.9%
Validation accuracy: 82.2%
Test accuracy: 88.2%


##  Logistic Regression  with SGD, 1-hidden layer neural network with rectified linear unit

In [0]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  

  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  
  
  # Training computation.
  logits = tf.matmul(hidden1, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  l2_regularizer = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)
  loss += 10e-4 * l2_regularizer
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [86]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 602.629822
Minibatch accuracy: 12.5%
Validation accuracy: 35.0%
Minibatch loss at step 500: 197.144058
Minibatch accuracy: 77.3%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 114.184151
Minibatch accuracy: 81.2%
Validation accuracy: 82.6%
Minibatch loss at step 1500: 68.517105
Minibatch accuracy: 86.7%
Validation accuracy: 83.9%
Minibatch loss at step 2000: 41.573666
Minibatch accuracy: 89.1%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.087870
Minibatch accuracy: 89.1%
Validation accuracy: 86.8%
Minibatch loss at step 3000: 15.513335
Minibatch accuracy: 88.3%
Validation accuracy: 87.2%
Test accuracy: 92.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [0]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  

  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  
  
  # Training computation.
  logits = tf.matmul(hidden1, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  l2_regularizer = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)
  loss += 10e-4 * l2_regularizer
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [53]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 705.427368
Minibatch accuracy: 0.0%
Validation accuracy: 11.6%
Minibatch loss at step 500: 150239.437500
Minibatch accuracy: 50.0%
Validation accuracy: 44.9%
Minibatch loss at step 1000: 69008.484375
Minibatch accuracy: 60.0%
Validation accuracy: 53.7%
Minibatch loss at step 1500: 200728.703125
Minibatch accuracy: 70.0%
Validation accuracy: 61.3%
Minibatch loss at step 2000: 201098.250000
Minibatch accuracy: 40.0%
Validation accuracy: 50.7%
Minibatch loss at step 2500: 415705.500000
Minibatch accuracy: 50.0%
Validation accuracy: 48.9%
Minibatch loss at step 3000: 94442.453125
Minibatch accuracy: 60.0%
Validation accuracy: 51.7%
Test accuracy: 55.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [60]:

def get_dataset_hash(dataset):
  return [
          sha1(dataset_instance).digest() 
          for dataset_instance in dataset
         ]

train_dataset_hash = get_dataset_hash(train_dataset)
valid_dataset_hash = get_dataset_hash(valid_dataset)
test_dataset_hash = get_dataset_hash(test_dataset)

duplicates_in_train_and_valid_dataset = np.intersect1d(train_dataset_hash, valid_dataset_hash)
duplicates_in_train_and_test_dataset = np.intersect1d(train_dataset_hash, test_dataset_hash)
duplicates_in_valid_and_test_dataset = np.intersect1d(test_dataset_hash, valid_dataset_hash)

duplicates = np.hstack(
                       (duplicates_in_train_and_valid_dataset,
                        duplicates_in_train_and_test_dataset,
                        duplicates_in_valid_and_test_dataset)
                      )

def get_sanitize(dataset, dataset_hash, exclude_hash):
  return np.array([
                   dataset[index] for index in 
                   np.arange(dataset.shape[0]) if
                   dataset_hash[index] not in 
                   exclude_hash
                 ])

sanitized_valid_dataset = get_sanitize(valid_dataset, valid_dataset_hash, duplicates)
sanitized_test_dataset = get_sanitize(test_dataset, test_dataset_hash, duplicates)

print ('original valid dataset shape',  valid_dataset.shape)
print ('sanitized valid dataset shape', sanitized_valid_dataset.shape)
print ('original test dataset shape',  test_dataset.shape)
print ('sanitized test dataset shape', sanitized_test_dataset.shape)



original valid dataset shape (10000, 784)
sanitized valid dataset shape (8900, 784)
original test dataset shape (10000, 784)
sanitized test dataset shape (8723, 784)


In [0]:
batch_size = 128

hidden_nodes_1 = 1024
hidden_nodes_2 = 512
hidden_nodes_3 = 128
#hidden_nodes_4 = 64

hidden_stddev_1 = np.sqrt(2.0/784) 
hidden_stddev_2 = np.sqrt(2.0/hidden_nodes_1)
hidden_stddev_3 = np.sqrt(2.0/hidden_nodes_2)
output_stddev   = np.sqrt(2.0/hidden_nodes_3)


regu = 10e-4 

keep_prob_1 = 0.5
keep_prob_2 = 0.6
keep_prob_3 = 0.7

deep_graph = tf.Graph()
with deep_graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  
  # first hidden layer
  hidden_weights_1 = tf.Variable(
    tf.truncated_normal(
      [image_size * image_size, hidden_nodes_1], stddev=hidden_stddev_1)) #
  hidden_biases_1 = tf.Variable(tf.zeros([hidden_nodes_1]))
  hidden_1 = tf.nn.dropout(
    tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights_1) + hidden_biases_1), keep_prob_1)
  
  # second hidden layer
  hidden_weights_2 = tf.Variable(
    tf.truncated_normal(
      [hidden_nodes_1, hidden_nodes_2], stddev=hidden_stddev_2)) #
  hidden_biases_2 = tf.Variable(tf.zeros([hidden_nodes_2]))
  hidden_2 = tf.nn.dropout(
    tf.nn.relu(tf.matmul(hidden_1, hidden_weights_2) + hidden_biases_2), keep_prob_2)
  
  # third hidden layer
  hidden_weights_3 = tf.Variable(
    tf.truncated_normal(
      [hidden_nodes_2, hidden_nodes_3], stddev=hidden_stddev_3)) #
  hidden_biases_3 = tf.Variable(tf.zeros([hidden_nodes_3]))
  hidden_3 = tf.nn.dropout(
    tf.nn.relu(tf.matmul(hidden_2, hidden_weights_3) + hidden_biases_3), keep_prob_3)
  
  # output layer
  output_weights = tf.Variable(
    tf.truncated_normal(
      [hidden_nodes_3, num_labels], stddev=output_stddev))#
  output_biases = tf.Variable(tf.zeros([num_labels]))
  
  
  # Training computation.
  
  logits = tf.matmul(hidden_3, output_weights) + output_biases  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
 
  loss += regu * (tf.nn.l2_loss(hidden_weights_1) +
                  tf.nn.l2_loss(hidden_weights_2) +
                  tf.nn.l2_loss(hidden_weights_3) +
                  tf.nn.l2_loss(output_weights))

  
  
  # Learn with exponential rate decay.
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.5
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 100000, 0.95, staircase=True)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  train_prediction = tf.nn.softmax(logits)
  

  
  # Predictions for the training, validation, and test data. 
  # Validation prediction step.
  validation_hidden_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights_1) + hidden_biases_1)
  validation_hidden_2 = tf.nn.relu(tf.matmul(validation_hidden_1, hidden_weights_2) + hidden_biases_2)
  validation_hidden_3 = tf.nn.relu(tf.matmul(validation_hidden_2, hidden_weights_3) + hidden_biases_3)
  validation_logits = tf.matmul(validation_hidden_3, output_weights) + output_biases
  validation_prediction = tf.nn.softmax(validation_logits)

  # Test prediction step.  
  test_hidden_1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights_1) + hidden_biases_1)
  test_hidden_2 = tf.nn.relu(tf.matmul(test_hidden_1, hidden_weights_2)+ hidden_biases_2)
  test_hidden_3 = tf.nn.relu(tf.matmul(test_hidden_2, hidden_weights_3) + hidden_biases_3)
  test_logits = tf.matmul(test_hidden_3, output_weights) + output_biases
  test_prediction = tf.nn.softmax(test_logits)
  
  
  

In [0]:
num_steps = 20001

with tf.Session(graph=deep_graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy( validation_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  
  

Initialized
Minibatch loss at step 0: 3.914595
Minibatch accuracy: 9.4%
Validation accuracy: 12.0%
Minibatch loss at step 500: 1.609692
Minibatch accuracy: 79.7%
Validation accuracy: 84.5%
Minibatch loss at step 1000: 1.150748
Minibatch accuracy: 83.6%
Validation accuracy: 85.8%
Minibatch loss at step 1500: 0.831414
Minibatch accuracy: 89.1%
Validation accuracy: 86.6%
Minibatch loss at step 2000: 0.808347
Minibatch accuracy: 85.9%
Validation accuracy: 86.6%
Minibatch loss at step 2500: 0.574893
Minibatch accuracy: 89.8%
Validation accuracy: 86.8%
Minibatch loss at step 3000: 0.736293
Minibatch accuracy: 84.4%
Validation accuracy: 87.1%
Minibatch loss at step 3500: 0.636759
Minibatch accuracy: 85.9%
Validation accuracy: 86.8%
Minibatch loss at step 4000: 0.549261
Minibatch accuracy: 91.4%
Validation accuracy: 87.4%



Test accuracy: 92.6%

